In [1]:
import cv2
import os
import numpy as np
import keras


data_path='emotions'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)


{'angry': 0, 'happy': 1, 'sad': 2}
['angry', 'happy', 'sad']
[0, 1, 2]


In [2]:
img_size=100
data=[]
target=[]

facedata = "haarcascade_frontalface_alt.xml"
cascade = cv2.CascadeClassifier(facedata)


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        faces = cascade.detectMultiScale(img) 
        try:
            for f in faces:
                x, y, w, h = [v for v in f]
                sub_face = img[y:y + h, x:x + w]
                gray=cv2.cvtColor(sub_face,cv2.COLOR_BGR2GRAY)           
                #Coverting the image into gray scale
                resized=cv2.resize(gray,(img_size,img_size))
                data.append(resized)
                target.append(label_dict[category])
        except Exception as e:
            print('Exception:',e)
    

In [ ]:
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)
from keras.utils import np_utils
new_target=np_utils.to_categorical(target)

In [4]:
np.save(r'C:\Users\91965\emotion based music player\emotions\data',data)
np.save(r'C:\Users\91965\emotion based music player\emotions\target',new_target)

In [5]:
data=np.load(r'C:\Users\91965\emotion based music player\emotions\data.npy')
target=np.load(r'C:\Users\91965\emotion based music player\emotions\target.npy')

In [6]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
model.add(Dense(3,activation='softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 200)       2000      
                                                                 
 activation (Activation)     (None, 98, 98, 200)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 200)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 100)       180100    
                                                                 
 activation_1 (Activation)   (None, 47, 47, 100)       0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 100)      0         
 2D)                                                    

In [7]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [8]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [9]:
print(train_data.shape)
print(train_target.shape)

(5193, 100, 100, 1)
(5193, 3)


In [10]:
checkpoint = ModelCheckpoint('./training/model{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto') 
history=model.fit(train_data,train_target,epochs=55,callbacks=checkpoint,validation_split=0.2)



Epoch 1/55
130/130 [==============================] - 125s 952ms/step - loss: 1.0963 - accuracy: 0.3893 - val_loss: 1.0918 - val_accuracy: 0.4187
Epoch 2/55
130/130 [==============================] - 128s 985ms/step - loss: 1.0624 - accuracy: 0.4458 - val_loss: 1.0232 - val_accuracy: 0.4928
Epoch 3/55
130/130 [==============================] - 121s 932ms/step - loss: 0.9467 - accuracy: 0.5272 - val_loss: 0.8700 - val_accuracy: 0.5602
Epoch 4/55
130/130 [==============================] - 122s 943ms/step - loss: 0.8704 - accuracy: 0.5821 - val_loss: 0.8229 - val_accuracy: 0.6352
Epoch 5/55
130/130 [==============================] - 117s 902ms/step - loss: 0.8376 - accuracy: 0.6122 - val_loss: 0.8226 - val_accuracy: 0.6218
Epoch 6/55
130/130 [==============================] - 125s 963ms/step - loss: 0.7916 - accuracy: 0.6305 - val_loss: 0.7842 - val_accuracy: 0.6323
Epoch 7/55
130/130 [==============================] - 129s 993ms/step - loss: 0.7689 - accuracy: 0.6558 - val_loss: 0.8211 -

Epoch 48/55
130/130 [==============================] - 131s 1s/step - loss: 0.3055 - accuracy: 0.8784 - val_loss: 0.8384 - val_accuracy: 0.6776
Epoch 49/55
130/130 [==============================] - 141s 1s/step - loss: 0.3145 - accuracy: 0.8724 - val_loss: 0.8565 - val_accuracy: 0.6728
Epoch 50/55
130/130 [==============================] - 122s 940ms/step - loss: 0.2914 - accuracy: 0.8885 - val_loss: 0.9068 - val_accuracy: 0.6641
Epoch 51/55
130/130 [==============================] - 125s 959ms/step - loss: 0.2902 - accuracy: 0.8857 - val_loss: 0.9116 - val_accuracy: 0.6660
Epoch 52/55
130/130 [==============================] - 122s 940ms/step - loss: 0.2787 - accuracy: 0.8917 - val_loss: 0.9211 - val_accuracy: 0.6641
Epoch 53/55
130/130 [==============================] - 126s 966ms/step - loss: 0.2731 - accuracy: 0.8948 - val_loss: 0.9730 - val_accuracy: 0.6843
Epoch 54/55
130/130 [==============================] - 124s 958ms/step - loss: 0.2752 - accuracy: 0.8953 - val_loss: 0.9237 

In [10]:
model.save("my_model")

INFO:tensorflow:Assets written to: my_model\assets


In [11]:
reconstructed_model = keras.models.load_model("my_model")


In [12]:
print(reconstructed_model.evaluate(test_data,test_target))
print(test_data.shape)
print(test_target.shape)


19/19 [==============================] - 4s 212ms/step - loss: 1.0999 - accuracy: 0.3253
[1.0999375581741333, 0.3252595067024231]
(578, 100, 100, 1)
(578, 3)


In [13]:
prediction=reconstructed_model.predict(test_data)

In [14]:
label=np.argmax(prediction,axis=1)
print(label)

[0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0
 2 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0
 0 0 0 2 0 0 0 2 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 2 0 0 0 0 0 0
 2 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 2 0 0 0 0
 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 2 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 2 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2
 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 2 2
 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [15]:
print(test_target)

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [16]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
labels_dict={0:'Angry',1:'happy',2:'Sad'}

color_dict={0:(0,0,255),1:(0,255,0),2:(255,0,0)}

In [4]:
cap=cv2.VideoCapture(0)
currentframe = 0
import random
import os
import IPython.display as ipd
while(True):
    
    ret,img=cap.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,3)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=reconstructed_model.predict(reshaped)
        label=np.argmax(result,axis=1)[0]
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
  
    #out.write(img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
    if ret==False:
        break
        
cv2.destroyAllWindows()
cap.release()
if labels_dict[label]  == 'angry':
    randomfile = random.choice(os.listdir(r"C:\Users\91965\project\songs\angry"))
    print('You are angry !!!!')
    file=os.path.join(r"C:\Users\91965\project\songs\angry",randomfile)
if labels_dict[label]  == 'happy':
    randomfile = random.choice(os.listdir(r"C:\Users\91965\project\songs\happy"))
    print('You are smiling :') 
    file=os.path.join(r"C:\Users\91965\project\songs\happy",randomfile)
    print(file)
if labels_dict[label]  == 'Sad':
    randomfile = random.choice(os.listdir(r"C:\Users\91965\project\songs\sad"))
    print('You are sad,dont worry:')
    file=os.path.join(r"C:\Users\91965\project\songs\sad",randomfile)
ipd.Audio(file) 

NameError: name 'face_clsfr' is not defined

In [43]:
print(labels_dict[label] )

happy
